In [1]:
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import glob
import re

In [2]:
folder_paths = ['/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_094824', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_104942', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_102335']

chlor_a_data_list = []

for folder_path in folder_paths:
    print("new folder")
    nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
    for file_name in nc_files:
        file_path = os.path.join(folder_path, file_name)
        
        dataset = xr.open_dataset(file_path)

        chlor_a_data_list.append(dataset['chlor_a'])

        dataset.close()

print("went through all files")

new folder
new folder
new folder
went through all files


In [12]:
chunk_size = 20 

concatenated_chlor_a = None

for i in range(0, len(chlor_a_data_list), chunk_size):
    chunk = chlor_a_data_list[i:i+chunk_size]
    chunk_concat = xr.concat(chunk, dim='time')
    
    if concatenated_chlor_a is None:
        concatenated_chlor_a = chunk_concat
    else:
        concatenated_chlor_a = xr.concat([concatenated_chlor_a, chunk_concat], dim='time')




Processed chunk 1 of 18
Processed chunk 2 of 18
Processed chunk 3 of 18
Processed chunk 4 of 18
Processed chunk 5 of 18
Processed chunk 6 of 18
Processed chunk 7 of 18
Processed chunk 8 of 18
Processed chunk 9 of 18
Processed chunk 10 of 18
Processed chunk 11 of 18
Processed chunk 12 of 18
Processed chunk 13 of 18
Processed chunk 14 of 18
Processed chunk 15 of 18
Processed chunk 16 of 18
Processed chunk 17 of 18
Processed chunk 18 of 18


In [13]:
output_file1 = '/cluster/home/maikents/satellite_data/chunks/chlor3.nc'
concatenated_chlor_a.to_netcdf(output_file1)

In [21]:
import xarray as xr

file_paths = [
    '/cluster/home/maikents/satellite_data/chunks/chlor1.nc',
    '/cluster/home/maikents/satellite_data/chunks/chlor2.nc',
    '/cluster/home/maikents/satellite_data/chunks/chlor3.nc'
]

concatenated_chlor_a = xr.open_mfdataset(file_paths, combine='nested', concat_dim='time')

print(concatenated_chlor_a)

<xarray.Dataset> Size: 161GB
Dimensions:  (time: 1081, lat: 4320, lon: 8640)
Coordinates:
  * lat      (lat) float32 17kB 89.98 89.94 89.9 89.85 ... -89.9 -89.94 -89.98
  * lon      (lon) float32 35kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: time
Data variables:
    chlor_a  (time, lat, lon) float32 161GB dask.array<chunksize=(23, 288, 576), meta=np.ndarray>


In [ ]:
concatenated_chlor_a.to_netcdf('/cluster/home/maikents/satellite_data/concatenated_chlor_a')

In [16]:
import xarray as xr
#denne!!
# Assuming concatenated_chlor_a is the concatenated chlorophyll data across time

# Calculate the mean across the time dimension
mean_chl = concatenated_chlor_a.mean(dim='time')

# Calculate the 10th percentile (min) across the time dimension
percentile_10 = concatenated_chlor_a.quantile(0.1, dim='time')

# Calculate the 90th percentile (max) across the time dimension
percentile_90 = concatenated_chlor_a.quantile(0.9, dim='time')

# Ensure that all variables (mean_chl, percentile_10, percentile_90) have the same coordinates
# Extract the coordinates from the original dataset
coords = {'lat': concatenated_chlor_a['lat'], 'lon': concatenated_chlor_a['lon']}

# Create a new xarray Dataset to hold the results
features = xr.Dataset({
    'mean_chl': (['lat', 'lon'], mean_chl),
    'percentile_10': (['lat', 'lon'], percentile_10),
    'percentile_90': (['lat', 'lon'], percentile_90)
}, coords=coords)

# Print the dataset structure to verify it contains the correct data
print(features)

# Optionally, save the dataset to a new NetCDF file for future use
features.to_netcdf('/cluster/home/maikents/satellite_data/satellite_features.nc')

print("Features (mean, 10th percentile, 90th percentile) saved")


/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,

KeyboardInterrupt

